# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [3]:
def expected_return(P, rf, mkt_rp):
    total_amount_invested = P['Amount Invested'].sum()
    beta = 0.0
    
    for i in range(len(P)):
        x = P['Amount Invested'][i] / total_amount_invested
        beta += x * P['Beta'][i]
       
    exp_rp = rf + beta * mkt_rp
        
    return exp_rp

#expected_return(portfolio, 0.02, 0.05)

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [4]:
def volatility(P, cor):
    total_amount_invested = P['Amount Invested'].sum()
    var_rp = 0.0
    P['x'] = P['Amount Invested'] / total_amount_invested
    
    for i in range(len(P)):
        var_rp += (P['x'][i]**2) * (P['Volatility'][i]**2)
        for j in range(i + 1, len(P)): 
            var_rp += 2 * P['x'][i] * P['x'][j] * P['Volatility'][i] * P['Volatility'][j] * cor
            
    SD = np.sqrt(var_rp)
    
    return SD

#volatility(portfolio, -0.2)

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [2]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    beta = vol / mkt_vol
    exp_rmkt = mkt_rp + rf
    exp_r = rf + beta * (exp_rmkt - rf)
    
    return exp_r

#expected_return2(0.02, 0.05, 0.16, 0.2408)

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [3]:
def CAPM_beta(return_path, factors_path):
    # Read return data
    return_data = pd.read_csv(return_path)
    
    # Read factors data
    factors_data = pd.read_csv(factors_path)
    
    factors_data['RF'] = factors_data['RF'] / 100
    factors_data['Mkt-RF'] = factors_data['Mkt-RF'] / 100
    
    # Assuming 'return_data' is your DataFrame
    return_data['date'] = return_data['year'].astype(str) + return_data['month'].astype(str).str.zfill(2)
    
    # Convert 'date' column to int64 in both dataframes
    return_data['date'] = return_data['date'].astype(int)
    factors_data['date'] = factors_data['date'].astype(int)
    
    # Merge return_data with factors_data based on 'CUSIP' and 'date'
    merged_data = pd.merge(return_data, factors_data, on = 'date')
    
    # Calculate expected return and CAPM beta for each CUSIP
    grouped_data = merged_data.groupby('CUSIP').apply(lambda group: pd.Series({
        "CUSIP": group['CUSIP'].iloc[0],
        "EXP_RETURN": group['RET'].mean(),
        "BETA": (group['RET'] - group['RF']).cov(group['Mkt-RF'] - group['RF']) / (group['Mkt-RF'] - group['RF']).var()
    }))
    
    return grouped_data.reset_index(drop=True)


return_path = '../../datafiles/A7/monthlycrsp_mod7.csv'
factors_path = '../../datafiles/A7/factors.CSV'

# Takes a while (~10 min)
data = CAPM_beta(return_path, factors_path)
data

CUSIP  EXP_RETURN       BETA
0  36020.0   -0.032023  -0.528747
1  36021.0    0.028259  -1.123571
2  36022.0    0.048830  -5.171334
3  36023.0    0.089881  10.183967

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [7]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.
    
    # Extract the beta and expected return from the data
    beta_values = data['BETA']
    expected_return_values = data['EXP_RETURN']

    # Fit a linear regression model using np.polyfit
    slope, intercept = np.polyfit(beta_values, expected_return_values, 1)

    return slope

#slope(CAPM_beta('monthlycrsp_mod7.csv', 'F-F_Research_Data_Factors.CSV'))